### Extracting and Renaming

In [1]:
import re
import tarfile
from pathlib import Path
import pandas as pd
import shutil
import os

In [68]:
FPS = 15
# directory= '14-3-2024_#15_INDIVIDUAL_(18)' #10 chunks
directory= '8-5-2024_#18_INDIVIDUAL_[12]'     #7 chunks  
BASE_PATH = f"/home/liubov/Desktop/BNF/work/{directory}/PosesDir"
base_path = Path(BASE_PATH)

# Find all chunk folders
chunk_dirs = sorted(base_path.glob("chunk_*"))
print(f"Found {len(chunk_dirs)} chunk folders:")
for cd in chunk_dirs:
    print(f"  - {cd.name}")

Found 7 chunk folders:
  - chunk_00_00000-00120s
  - chunk_01_00120-00240s
  - chunk_02_00240-00360s
  - chunk_03_00360-00480s
  - chunk_04_00480-00600s
  - chunk_05_00600-00720s
  - chunk_06_00720-00785s


In [3]:
total_renamed = 0

for chunk_dir in chunk_dirs:
    # Parse start/end seconds from folder name
    m = re.search(r"chunk_\d+_(\d+)-(\d+)s$", chunk_dir.name)
    if not m:
        print(f"[SKIP] Can't parse chunk range from: {chunk_dir}")
        continue
    start_sec, end_sec = map(int, m.groups())
    abs_base = start_sec * FPS

    print(f"\n=== Processing {chunk_dir} ({start_sec:05}-{end_sec:05}s) ===")

    # Find all tar.gz files in this chunk folder
    tar_files = sorted(chunk_dir.glob("*.tar.gz"))
    if not tar_files:
        print("  No tar.gz files found.")
        continue

    for tar_file in tar_files:
        print(f"  Extracting {tar_file.name}...")

        extract_dir = tar_file.with_name(f"{tar_file.stem}_extracted")
        extract_dir.mkdir(exist_ok=True)

        try:
            with tarfile.open(tar_file, "r:gz") as tar:
                for member in tar.getmembers():
                    member.path = Path(member.name).name  # flatten
                    tar.extract(member, path=extract_dir)
        except Exception as e:
            print(f"  Error extracting {tar_file.name}: {e}")
            continue

        # Rename JSON files
        json_files = list(extract_dir.rglob("*.json"))
        if not json_files:
            print("   No JSON files found.")
            continue

        # video_id can be derived from tar file name or fixed
        video_id = tar_file.stem.split('_')[0]

        for jf in json_files:
            stem = jf.stem
            mnum = re.search(r"(\d+)$", stem)
            if not mnum:
                print(f"    Skipping (no trailing number): {jf.name}")
                continue

            local_idx = int(mnum.group(1))
            abs_idx = abs_base + local_idx

            new_name = f"id_{video_id}_{start_sec:05}-{end_sec:05}s_{abs_idx:04}.json"
            new_path = jf.with_name(new_name)

            if jf.name == new_name:
                continue

            if new_path.exists():
                print(f"    Warning: target exists, skipping: {new_name}")
                continue

            jf.rename(new_path)
            print(f"    {jf.name} -> {new_name}")
            total_renamed += 1


=== Processing /home/liubov/Desktop/BNF/work/14-3-2024_#15_INDIVIDUAL_(18)/PosesDir/chunk_00_00000-00120s (00000-00120s) ===
  Extracting 1.tar.gz...
     00000000001089.json -> id_1.tar_00000-00120s_1089.json
     00000000000875.json -> id_1.tar_00000-00120s_0875.json
     00000000000947.json -> id_1.tar_00000-00120s_0947.json
     00000000000741.json -> id_1.tar_00000-00120s_0741.json
     00000000000261.json -> id_1.tar_00000-00120s_0261.json
     00000000000950.json -> id_1.tar_00000-00120s_0950.json
     00000000001301.json -> id_1.tar_00000-00120s_1301.json
     00000000001145.json -> id_1.tar_00000-00120s_1145.json
     00000000001143.json -> id_1.tar_00000-00120s_1143.json
     00000000001619.json -> id_1.tar_00000-00120s_1619.json
     00000000001641.json -> id_1.tar_00000-00120s_1641.json
     00000000001139.json -> id_1.tar_00000-00120s_1139.json
     00000000000184.json -> id_1.tar_00000-00120s_0184.json
     00000000000567.json -> id_1.tar_00000-00120s_0567.json
     0000

     00000000001089.json -> id_2.tar_00000-00120s_1089.json
     00000000000875.json -> id_2.tar_00000-00120s_0875.json
     00000000000947.json -> id_2.tar_00000-00120s_0947.json
     00000000000741.json -> id_2.tar_00000-00120s_0741.json
     00000000000261.json -> id_2.tar_00000-00120s_0261.json
     00000000000950.json -> id_2.tar_00000-00120s_0950.json
     00000000001301.json -> id_2.tar_00000-00120s_1301.json
     00000000001145.json -> id_2.tar_00000-00120s_1145.json
     00000000001143.json -> id_2.tar_00000-00120s_1143.json
     00000000001619.json -> id_2.tar_00000-00120s_1619.json
     00000000001641.json -> id_2.tar_00000-00120s_1641.json
     00000000001139.json -> id_2.tar_00000-00120s_1139.json
     00000000000184.json -> id_2.tar_00000-00120s_0184.json
     00000000000567.json -> id_2.tar_00000-00120s_0567.json
     00000000001416.json -> id_2.tar_00000-00120s_1416.json
     00000000000015.json -> id_2.tar_00000-00120s_0015.json
     00000000000442.json -> id_2.tar_000

     00000000001089.json -> id_1.tar_00240-00360s_4689.json
     00000000000875.json -> id_1.tar_00240-00360s_4475.json
     00000000000947.json -> id_1.tar_00240-00360s_4547.json
     00000000000741.json -> id_1.tar_00240-00360s_4341.json
     00000000000261.json -> id_1.tar_00240-00360s_3861.json
     00000000000950.json -> id_1.tar_00240-00360s_4550.json
     00000000001301.json -> id_1.tar_00240-00360s_4901.json
     00000000001145.json -> id_1.tar_00240-00360s_4745.json
     00000000001143.json -> id_1.tar_00240-00360s_4743.json
     00000000001619.json -> id_1.tar_00240-00360s_5219.json
     00000000001641.json -> id_1.tar_00240-00360s_5241.json
     00000000001139.json -> id_1.tar_00240-00360s_4739.json
     00000000000184.json -> id_1.tar_00240-00360s_3784.json
     00000000000567.json -> id_1.tar_00240-00360s_4167.json
     00000000001416.json -> id_1.tar_00240-00360s_5016.json
     00000000000015.json -> id_1.tar_00240-00360s_3615.json
     00000000000442.json -> id_1.tar_002

     00000000001089.json -> id_2.tar_00240-00360s_4689.json
     00000000000875.json -> id_2.tar_00240-00360s_4475.json
     00000000000947.json -> id_2.tar_00240-00360s_4547.json
     00000000000741.json -> id_2.tar_00240-00360s_4341.json
     00000000000261.json -> id_2.tar_00240-00360s_3861.json
     00000000000950.json -> id_2.tar_00240-00360s_4550.json
     00000000001301.json -> id_2.tar_00240-00360s_4901.json
     00000000001145.json -> id_2.tar_00240-00360s_4745.json
     00000000001143.json -> id_2.tar_00240-00360s_4743.json
     00000000001619.json -> id_2.tar_00240-00360s_5219.json
     00000000001641.json -> id_2.tar_00240-00360s_5241.json
     00000000001139.json -> id_2.tar_00240-00360s_4739.json
     00000000000184.json -> id_2.tar_00240-00360s_3784.json
     00000000000567.json -> id_2.tar_00240-00360s_4167.json
     00000000001416.json -> id_2.tar_00240-00360s_5016.json
     00000000000015.json -> id_2.tar_00240-00360s_3615.json
     00000000000442.json -> id_2.tar_002

     00000000001089.json -> id_1.tar_00360-00480s_6489.json
     00000000000875.json -> id_1.tar_00360-00480s_6275.json
     00000000000947.json -> id_1.tar_00360-00480s_6347.json
     00000000000741.json -> id_1.tar_00360-00480s_6141.json
     00000000000261.json -> id_1.tar_00360-00480s_5661.json
     00000000000950.json -> id_1.tar_00360-00480s_6350.json
     00000000001301.json -> id_1.tar_00360-00480s_6701.json
     00000000001145.json -> id_1.tar_00360-00480s_6545.json
     00000000001143.json -> id_1.tar_00360-00480s_6543.json
     00000000001619.json -> id_1.tar_00360-00480s_7019.json
     00000000001641.json -> id_1.tar_00360-00480s_7041.json
     00000000001139.json -> id_1.tar_00360-00480s_6539.json
     00000000000184.json -> id_1.tar_00360-00480s_5584.json
     00000000000567.json -> id_1.tar_00360-00480s_5967.json
     00000000001416.json -> id_1.tar_00360-00480s_6816.json
     00000000000015.json -> id_1.tar_00360-00480s_5415.json
     00000000000442.json -> id_1.tar_003

...

     00000000001089.json -> id_1.tar_01560-01680s_24489.json
     00000000000875.json -> id_1.tar_01560-01680s_24275.json
     00000000000947.json -> id_1.tar_01560-01680s_24347.json
     00000000000741.json -> id_1.tar_01560-01680s_24141.json
     00000000000261.json -> id_1.tar_01560-01680s_23661.json
     00000000000950.json -> id_1.tar_01560-01680s_24350.json
     00000000001301.json -> id_1.tar_01560-01680s_24701.json
     00000000001145.json -> id_1.tar_01560-01680s_24545.json
     00000000001143.json -> id_1.tar_01560-01680s_24543.json
     00000000001619.json -> id_1.tar_01560-01680s_25019.json
     00000000001641.json -> id_1.tar_01560-01680s_25041.json
     00000000001139.json -> id_1.tar_01560-01680s_24539.json
     00000000000184.json -> id_1.tar_01560-01680s_23584.json
     00000000000567.json -> id_1.tar_01560-01680s_23967.json
     00000000001416.json -> id_1.tar_01560-01680s_24816.json
     00000000000015.json -> id_1.tar_01560-01680s_23415.json
     00000000000442.json

     00000000001089.json -> id_2.tar_01560-01680s_24489.json
     00000000000875.json -> id_2.tar_01560-01680s_24275.json
     00000000000947.json -> id_2.tar_01560-01680s_24347.json
     00000000000741.json -> id_2.tar_01560-01680s_24141.json
     00000000000261.json -> id_2.tar_01560-01680s_23661.json
     00000000000950.json -> id_2.tar_01560-01680s_24350.json
     00000000001301.json -> id_2.tar_01560-01680s_24701.json
     00000000001145.json -> id_2.tar_01560-01680s_24545.json
     00000000001143.json -> id_2.tar_01560-01680s_24543.json
     00000000001619.json -> id_2.tar_01560-01680s_25019.json
     00000000001641.json -> id_2.tar_01560-01680s_25041.json
     00000000001139.json -> id_2.tar_01560-01680s_24539.json
     00000000000184.json -> id_2.tar_01560-01680s_23584.json
     00000000000567.json -> id_2.tar_01560-01680s_23967.json
     00000000001416.json -> id_2.tar_01560-01680s_24816.json
     00000000000015.json -> id_2.tar_01560-01680s_23415.json
     00000000000442.json

     00000000001089.json -> id_1.tar_01800-01920s_28089.json
     00000000000875.json -> id_1.tar_01800-01920s_27875.json
     00000000000947.json -> id_1.tar_01800-01920s_27947.json
     00000000000741.json -> id_1.tar_01800-01920s_27741.json
     00000000000261.json -> id_1.tar_01800-01920s_27261.json
     00000000000950.json -> id_1.tar_01800-01920s_27950.json
     00000000001301.json -> id_1.tar_01800-01920s_28301.json
     00000000001145.json -> id_1.tar_01800-01920s_28145.json
     00000000001143.json -> id_1.tar_01800-01920s_28143.json
     00000000001619.json -> id_1.tar_01800-01920s_28619.json
     00000000001641.json -> id_1.tar_01800-01920s_28641.json
     00000000001139.json -> id_1.tar_01800-01920s_28139.json
     00000000000184.json -> id_1.tar_01800-01920s_27184.json
     00000000000567.json -> id_1.tar_01800-01920s_27567.json
     00000000001416.json -> id_1.tar_01800-01920s_28416.json
     00000000000015.json -> id_1.tar_01800-01920s_27015.json
     00000000000442.json

     00000000001089.json -> id_2.tar_01800-01920s_28089.json
     00000000000875.json -> id_2.tar_01800-01920s_27875.json
     00000000000947.json -> id_2.tar_01800-01920s_27947.json
     00000000000741.json -> id_2.tar_01800-01920s_27741.json
     00000000000261.json -> id_2.tar_01800-01920s_27261.json
     00000000000950.json -> id_2.tar_01800-01920s_27950.json
     00000000001301.json -> id_2.tar_01800-01920s_28301.json
     00000000001145.json -> id_2.tar_01800-01920s_28145.json
     00000000001143.json -> id_2.tar_01800-01920s_28143.json
     00000000001619.json -> id_2.tar_01800-01920s_28619.json
     00000000001641.json -> id_2.tar_01800-01920s_28641.json
     00000000001139.json -> id_2.tar_01800-01920s_28139.json
     00000000000184.json -> id_2.tar_01800-01920s_27184.json
     00000000000567.json -> id_2.tar_01800-01920s_27567.json
     00000000001416.json -> id_2.tar_01800-01920s_28416.json
     00000000000015.json -> id_2.tar_01800-01920s_27015.json
     00000000000442.json

     00000000001089.json -> id_3.tar_01800-01920s_28089.json
     00000000000875.json -> id_3.tar_01800-01920s_27875.json
     00000000000947.json -> id_3.tar_01800-01920s_27947.json
     00000000000741.json -> id_3.tar_01800-01920s_27741.json
     00000000000261.json -> id_3.tar_01800-01920s_27261.json
     00000000000950.json -> id_3.tar_01800-01920s_27950.json
     00000000001301.json -> id_3.tar_01800-01920s_28301.json
     00000000001145.json -> id_3.tar_01800-01920s_28145.json
     00000000001143.json -> id_3.tar_01800-01920s_28143.json
     00000000001619.json -> id_3.tar_01800-01920s_28619.json
     00000000001641.json -> id_3.tar_01800-01920s_28641.json
     00000000001139.json -> id_3.tar_01800-01920s_28139.json
     00000000000184.json -> id_3.tar_01800-01920s_27184.json
     00000000000567.json -> id_3.tar_01800-01920s_27567.json
     00000000001416.json -> id_3.tar_01800-01920s_28416.json
     00000000000015.json -> id_3.tar_01800-01920s_27015.json
     00000000000442.json

In [4]:
print(f"\n Summary ")
print(f"Total JSON files renamed: {total_renamed}")


 Summary 
Total JSON files renamed: 41400


### Collect all json-files

In [5]:
import os
import shutil

# Base directory where all your folders are
base_dir = f'/home/liubov/Desktop/BNF/work/{directory}/PosesDir'

# Destination folder where all JSONs will be copied
dest_dir = os.path.join(base_dir, "all_jsons")
os.makedirs(dest_dir, exist_ok=True)

# Loop through all subdirectories
for root, dirs, files in os.walk(base_dir):
    # Skip the all_jsons directory itself
    if root == dest_dir:
        continue

    for file in files:
        if file.endswith(".json"):
            src_file = os.path.join(root, file)
            dst_file = os.path.join(dest_dir, file)

            # Skip if source == destination
            if os.path.abspath(src_file) == os.path.abspath(dst_file):
                continue

            shutil.copy2(src_file, dst_file)

print("All JSON files have been copied to:", dest_dir)

All JSON files have been copied to: /home/liubov/Desktop/BNF/work/14-3-2024_#15_INDIVIDUAL_(18)/PosesDir/all_jsons


In [6]:
import os
import shutil

# Base directory where all your folders are
base_dir = f'/home/liubov/Desktop/BNF/work/{directory}/PosesDir'

# Destination folder where all JSONs will be copied
dest_dir = os.path.join(base_dir, "all_jsons")
os.makedirs(dest_dir, exist_ok=True)

# Loop through all subdirectories
for root, dirs, files in os.walk(base_dir):
    # prevent walking into all_jsons
    if os.path.abspath(root) == os.path.abspath(dest_dir):
        continue

    for file in files:
        if file.endswith(".json"):
            src_file = os.path.join(root, file)
            dst_file = os.path.join(dest_dir, file)

            # extra safety: skip if src == dst
            if os.path.abspath(src_file) == os.path.abspath(dst_file):
                continue

            shutil.copy2(src_file, dst_file)

print("All JSON files have been copied to:", dest_dir)

All JSON files have been copied to: /home/liubov/Desktop/BNF/work/14-3-2024_#15_INDIVIDUAL_(18)/PosesDir/all_jsons


### Download and Preparing Mapping File

In [7]:
mapping_df=pd.read_csv(f'/home/liubov/Desktop/BNF/work/{directory}/first_id_mapping_camera_a.csv')

In [8]:
mapping_df.drop(columns='Unnamed: 0', inplace=True)

In [9]:
mapping_df

,frame_number,original_id,updated_id
0,5400,1,2
1,5400,2,1
2,7200,1,2
3,7200,2,1
4,9000,1,2
5,9000,2,1
6,23400,1,2
7,23400,2,1


In [10]:
all_jsons_dir = f"/home/liubov/Desktop/BNF/work/{directory}/PosesDir/all_jsons"

frame_numbers = []
pattern = re.compile(r'_(\d+)\.json$')  # captures the last number before .json

for file in os.listdir(all_jsons_dir):
    if file.endswith(".json"):
        match = pattern.search(file)
        if match:
            frame_numbers.append(int(match.group(1)))

if frame_numbers:
    max_frame = max(frame_numbers)
else:
    raise ValueError("No JSON files with frame numbers found!")

print("Max frame number found:", max_frame)

# Example: your existing mapping_df logic
mapping_df["next_frame_number"] = mapping_df.groupby("frame_number")["frame_number"].transform("max").shift(-3, fill_value=max_frame)

# Expand mapping_df
expanded = mapping_df.apply(
    lambda row: pd.DataFrame({
        "frame_number": range(row["frame_number"], row["next_frame_number"]),
        "original_id": row["original_id"],
        "updated_id": row["updated_id"]
    }), axis=1
)

expanded_df = pd.concat(expanded.tolist(), ignore_index=True)

print(expanded_df)

Max frame number found: 28799
       frame_number  original_id  updated_id
0              5400            1           2
1              5401            1           2
2              5402            1           2
3              5403            1           2
4              5404            1           2
...             ...          ...         ...
68392         28794            2           1
68393         28795            2           1
68394         28796            2           1
68395         28797            2           1
68396         28798            2           1

[68397 rows x 3 columns]


In [11]:
# expanded_df.tail(10)

In [12]:
# mapping_df.tail()

### Original ID to Updated ID Renaming

In [13]:
json_directory_path = f'/home/liubov/Desktop/BNF/work/{directory}/PosesDir/all_jsons'
json_dir = Path(json_directory_path)

json_directory_path_new = f'/home/liubov/Desktop/BNF/work/{directory}/PosesDir/new_jsons'

output_dir = Path(json_directory_path_new)
output_dir.mkdir(exist_ok=True)

json_files = list(json_dir.glob("*.json"))
print(f"Found {len(json_files)} JSON files")

Found 41401 JSON files


In [14]:
pattern = re.compile(r"^id_(\d+)\.tar_\d+-\d+s_(\d+)\.json$")

renamed_count = 0
copied_count = 0

for json_file in json_files:
    filename = json_file.name
    old_path = json_file

    match = pattern.match(filename)
    if match:
        try:
            original_id = int(match.group(1))
            frame_number = int(match.group(2))

            # Try to find mapping
            matching_rows = expanded_df[
                (expanded_df['frame_number'] == frame_number) &
                (expanded_df['original_id'] == original_id)
            ]

            if len(matching_rows) > 0:
                updated_id = matching_rows.iloc[0]['updated_id']
            else:
                print(f"  No mapping found for ID {original_id}, frame {frame_number} - using original ID")
                updated_id = original_id

            # Always generate new filename in the unified format
            new_filename = f"id_{updated_id}_{frame_number}.json"
            new_path = output_dir / new_filename

            if new_path.exists():
                print(f"  WARNING: Target file {new_filename} already exists!")
                continue

            shutil.copy2(str(old_path), str(new_path))

            if len(matching_rows) > 0:
                renamed_count += 1
            else:
                copied_count += 1

        except Exception as e:
            print(f"  Error processing file {filename}: {e}")
            continue

    else:
        # Doesn’t match pattern → fallback: just copy with same name (or make up format?)
        print(f"Skipping file (pattern mismatch): {filename}")
        new_path = output_dir / filename
        if not new_path.exists():
            shutil.copy2(str(old_path), str(new_path))
            copied_count += 1
        else:
            print(f"  WARNING: Target file {filename} already exists!")

print(f"\nCompleted! Renamed {renamed_count} files, copied {copied_count} files with original IDs.")


  No mapping found for ID 3, frame 28126 - using original ID
  No mapping found for ID 1, frame 4315 - using original ID
  No mapping found for ID 1, frame 892 - using original ID
  No mapping found for ID 2, frame 774 - using original ID
  No mapping found for ID 1, frame 4468 - using original ID
  No mapping found for ID 1, frame 3794 - using original ID
  No mapping found for ID 3, frame 27352 - using original ID
  No mapping found for ID 2, frame 1460 - using original ID
  No mapping found for ID 2, frame 935 - using original ID
  No mapping found for ID 1, frame 288 - using original ID
  No mapping found for ID 1, frame 345 - using original ID
  No mapping found for ID 3, frame 28223 - using original ID
  No mapping found for ID 3, frame 27959 - using original ID
  No mapping found for ID 3, frame 16267 - using original ID
  No mapping found for ID 3, frame 17244 - using original ID
  No mapping found for ID 3, frame 9917 - using original ID
  No mapping found for ID 3, frame 1046

  No mapping found for ID 1, frame 334 - using original ID
  No mapping found for ID 1, frame 715 - using original ID
  No mapping found for ID 3, frame 17408 - using original ID
  No mapping found for ID 3, frame 27158 - using original ID
  No mapping found for ID 3, frame 10156 - using original ID
  No mapping found for ID 3, frame 28003 - using original ID
  No mapping found for ID 1, frame 4488 - using original ID
  No mapping found for ID 2, frame 4720 - using original ID
  No mapping found for ID 3, frame 9443 - using original ID
  No mapping found for ID 2, frame 243 - using original ID
  No mapping found for ID 1, frame 4453 - using original ID
  No mapping found for ID 1, frame 43 - using original ID
  No mapping found for ID 3, frame 17722 - using original ID
  No mapping found for ID 1, frame 708 - using original ID
  No mapping found for ID 3, frame 16980 - using original ID
  No mapping found for ID 2, frame 4173 - using original ID
  No mapping found for ID 1, frame 819 -

  No mapping found for ID 1, frame 1639 - using original ID
  No mapping found for ID 3, frame 17232 - using original ID
  No mapping found for ID 3, frame 9160 - using original ID
  No mapping found for ID 1, frame 5173 - using original ID
  No mapping found for ID 3, frame 28278 - using original ID
  No mapping found for ID 1, frame 3658 - using original ID
  No mapping found for ID 3, frame 16809 - using original ID
  No mapping found for ID 3, frame 17161 - using original ID
  No mapping found for ID 2, frame 4230 - using original ID
  No mapping found for ID 1, frame 227 - using original ID
  No mapping found for ID 3, frame 16505 - using original ID
  No mapping found for ID 2, frame 448 - using original ID
  No mapping found for ID 3, frame 9186 - using original ID
  No mapping found for ID 1, frame 3726 - using original ID
  No mapping found for ID 1, frame 419 - using original ID
  No mapping found for ID 3, frame 17216 - using original ID
  No mapping found for ID 3, frame 16

  No mapping found for ID 2, frame 484 - using original ID
  No mapping found for ID 3, frame 9446 - using original ID
  No mapping found for ID 2, frame 1490 - using original ID
  No mapping found for ID 1, frame 4528 - using original ID
  No mapping found for ID 2, frame 4879 - using original ID
  No mapping found for ID 1, frame 3824 - using original ID
  No mapping found for ID 3, frame 9956 - using original ID
  No mapping found for ID 3, frame 28144 - using original ID
  No mapping found for ID 2, frame 404 - using original ID
  No mapping found for ID 1, frame 4221 - using original ID
  No mapping found for ID 1, frame 5081 - using original ID
  No mapping found for ID 3, frame 27705 - using original ID
  No mapping found for ID 3, frame 27377 - using original ID
  No mapping found for ID 1, frame 5033 - using original ID
  No mapping found for ID 1, frame 1495 - using original ID
  No mapping found for ID 2, frame 638 - using original ID
  No mapping found for ID 1, frame 3692 

  No mapping found for ID 3, frame 28099 - using original ID
  No mapping found for ID 1, frame 3911 - using original ID
  No mapping found for ID 2, frame 4770 - using original ID
  No mapping found for ID 2, frame 4951 - using original ID
  No mapping found for ID 1, frame 3623 - using original ID
  No mapping found for ID 3, frame 27613 - using original ID
  No mapping found for ID 1, frame 568 - using original ID
  No mapping found for ID 3, frame 28401 - using original ID
  No mapping found for ID 3, frame 17628 - using original ID
  No mapping found for ID 1, frame 4672 - using original ID
  No mapping found for ID 1, frame 141 - using original ID
  No mapping found for ID 1, frame 5112 - using original ID
  No mapping found for ID 3, frame 10767 - using original ID
  No mapping found for ID 2, frame 1797 - using original ID
  No mapping found for ID 1, frame 4951 - using original ID
  No mapping found for ID 2, frame 4911 - using original ID
  No mapping found for ID 1, frame 39

...

  No mapping found for ID 2, frame 1286 - using original ID
  No mapping found for ID 3, frame 9509 - using original ID
  No mapping found for ID 3, frame 17912 - using original ID
  No mapping found for ID 3, frame 16240 - using original ID
  No mapping found for ID 2, frame 4741 - using original ID
  No mapping found for ID 3, frame 9878 - using original ID
  No mapping found for ID 3, frame 9158 - using original ID
  No mapping found for ID 2, frame 672 - using original ID
  No mapping found for ID 3, frame 28421 - using original ID
  No mapping found for ID 2, frame 748 - using original ID
  No mapping found for ID 3, frame 28512 - using original ID
  No mapping found for ID 2, frame 3957 - using original ID
  No mapping found for ID 2, frame 1142 - using original ID
  No mapping found for ID 3, frame 27984 - using original ID
  No mapping found for ID 3, frame 27807 - using original ID
  No mapping found for ID 3, frame 10709 - using original ID
  No mapping found for ID 2, frame 

  No mapping found for ID 2, frame 4941 - using original ID
  No mapping found for ID 2, frame 5190 - using original ID
  No mapping found for ID 1, frame 4068 - using original ID
  No mapping found for ID 3, frame 28784 - using original ID
  No mapping found for ID 2, frame 3717 - using original ID
  No mapping found for ID 1, frame 636 - using original ID
  No mapping found for ID 1, frame 652 - using original ID
  No mapping found for ID 2, frame 3615 - using original ID
  No mapping found for ID 1, frame 5377 - using original ID
  No mapping found for ID 3, frame 10339 - using original ID
  No mapping found for ID 2, frame 3854 - using original ID
  No mapping found for ID 2, frame 3843 - using original ID
  No mapping found for ID 2, frame 1233 - using original ID
  No mapping found for ID 1, frame 4387 - using original ID
  No mapping found for ID 3, frame 27476 - using original ID
  No mapping found for ID 2, frame 4624 - using original ID
  No mapping found for ID 2, frame 1166

  No mapping found for ID 3, frame 16532 - using original ID
  No mapping found for ID 3, frame 17911 - using original ID
  No mapping found for ID 3, frame 9171 - using original ID
  No mapping found for ID 2, frame 3777 - using original ID
  No mapping found for ID 2, frame 350 - using original ID
  No mapping found for ID 3, frame 10430 - using original ID
  No mapping found for ID 3, frame 27296 - using original ID
  No mapping found for ID 1, frame 73 - using original ID
  No mapping found for ID 3, frame 16450 - using original ID
  No mapping found for ID 2, frame 671 - using original ID
  No mapping found for ID 1, frame 3918 - using original ID
  No mapping found for ID 3, frame 17426 - using original ID
  No mapping found for ID 2, frame 4550 - using original ID
  No mapping found for ID 3, frame 28721 - using original ID
  No mapping found for ID 3, frame 27601 - using original ID
  No mapping found for ID 1, frame 4150 - using original ID
  No mapping found for ID 3, frame 2

  No mapping found for ID 1, frame 4722 - using original ID
  No mapping found for ID 3, frame 17836 - using original ID
  No mapping found for ID 1, frame 4006 - using original ID
  No mapping found for ID 1, frame 600 - using original ID
  No mapping found for ID 2, frame 4349 - using original ID
  No mapping found for ID 1, frame 613 - using original ID
  No mapping found for ID 1, frame 151 - using original ID
  No mapping found for ID 1, frame 1660 - using original ID
  No mapping found for ID 3, frame 17601 - using original ID
  No mapping found for ID 3, frame 16238 - using original ID
  No mapping found for ID 2, frame 4930 - using original ID
  No mapping found for ID 2, frame 148 - using original ID
  No mapping found for ID 2, frame 1743 - using original ID
  No mapping found for ID 1, frame 1283 - using original ID
  No mapping found for ID 1, frame 772 - using original ID
  No mapping found for ID 3, frame 28716 - using original ID
  No mapping found for ID 3, frame 10004 

  No mapping found for ID 1, frame 4640 - using original ID
  No mapping found for ID 1, frame 1672 - using original ID
  No mapping found for ID 3, frame 27136 - using original ID
  No mapping found for ID 2, frame 1664 - using original ID
  No mapping found for ID 2, frame 1686 - using original ID
  No mapping found for ID 3, frame 16727 - using original ID
  No mapping found for ID 2, frame 5056 - using original ID
  No mapping found for ID 1, frame 4833 - using original ID
  No mapping found for ID 1, frame 4792 - using original ID
  No mapping found for ID 1, frame 5145 - using original ID
  No mapping found for ID 1, frame 1119 - using original ID
  No mapping found for ID 3, frame 27690 - using original ID
  No mapping found for ID 2, frame 1081 - using original ID
  No mapping found for ID 1, frame 5309 - using original ID
  No mapping found for ID 3, frame 17996 - using original ID
  No mapping found for ID 1, frame 3737 - using original ID
  No mapping found for ID 3, frame 2

### JSONs Merging

In [69]:
import json
folder_path = f'/home/liubov/Desktop/BNF/work/{directory}/PosesDir/new_jsons'
pattern = re.compile(r"id_\d+")
grouped_data = {}

for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        match = pattern.search(filename)
        if match:
            group = match.group()  # e.g., "id_1"
            grouped_data.setdefault(group, []).append(filename)

for group, files in grouped_data.items():
    files.sort()

    merged_data = []
    for filename in files:
        file_path = os.path.join(folder_path, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            merged_data.append(data)

    output_file = f"{folder_path}_merged_{group}.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(merged_data, f, indent=4, ensure_ascii=False)

    print(f"Merged {group} ({len(files)} files) -> {output_file}")

Merged id_1 (11785 files) -> /home/liubov/Desktop/BNF/work/8-5-2024_#18_INDIVIDUAL_[12]/PosesDir/new_jsons_merged_id_1.json
Merged id_2 (11785 files) -> /home/liubov/Desktop/BNF/work/8-5-2024_#18_INDIVIDUAL_[12]/PosesDir/new_jsons_merged_id_2.json
Merged id_3 (11785 files) -> /home/liubov/Desktop/BNF/work/8-5-2024_#18_INDIVIDUAL_[12]/PosesDir/new_jsons_merged_id_3.json
